In [1]:
# 피트 스톱 데이터(PitIn/PitOut) DB 업데이트 스크립트
import fastf1
import pandas as pd
from pymongo import MongoClient
import warnings

# 설정
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "f1_analytics_db"
TARGET_YEARS = [2023, 2024, 2025]  # 업데이트할 연도

# 캐시 및 경고 설정
fastf1.Cache.enable_cache(r'D:\f1_cache_temp') # 기존에 쓰던 캐시 경로 사용 권장
warnings.filterwarnings('ignore')

# DB 연결
client = MongoClient(MONGO_URI)
db = client[DB_NAME]
col_laps = db["laps"]

print(f"✅ DB 연결 성공: {DB_NAME}")

def update_pit_data(years):
    for year in years:
        try:
            # 스케줄 로드
            schedule = fastf1.get_event_schedule(year)
            completed_races = schedule[schedule['EventDate'] < pd.Timestamp.now()]

            print(f"\n🛠️ {year}년도 피트 데이터 업데이트 시작 ({len(completed_races)} 라운드)...")

            for _, row in completed_races.iterrows():
                round_num = row['RoundNumber']
                event_name = row['EventName']

                # Pre-Season 제외
                if round_num == 0: continue

                print(f"   Processing: R{round_num} {event_name}...", end=" ")

                try:
                    # 세션 로드 (랩 데이터만 필요하므로 telemetry=False)
                    session = fastf1.get_session(year, round_num, 'R')
                    session.load(telemetry=False, weather=False, messages=False)

                    laps = session.laps
                    update_count = 0

                    # 피트 스톱이 있는 랩만 필터링하여 업데이트
                    # (PitInTime이나 PitOutTime이 존재하는 경우)
                    pit_laps = laps.dropna(subset=['PitInTime', 'PitOutTime'], how='all')

                    for _, lap in pit_laps.iterrows():
                        # 업데이트할 필드 준비 (Timedelta -> Seconds 변환)
                        update_fields = {}
                        if pd.notna(lap['PitInTime']):
                            update_fields['PitInTime'] = lap['PitInTime'].total_seconds()
                        if pd.notna(lap['PitOutTime']):
                            update_fields['PitOutTime'] = lap['PitOutTime'].total_seconds()

                        if not update_fields: continue

                        # DB 업데이트 쿼리
                        # 키: Year, Round, SessionType, Driver, LapNumber
                        filter_query = {
                            "Year": int(year),
                            "Round": int(round_num),
                            "SessionType": "Race", # 또는 'R' (저장된 방식 확인)
                            "Driver": lap['Driver'],
                            "LapNumber": int(lap['LapNumber'])
                        }

                        result = col_laps.update_one(filter_query, {"$set": update_fields})
                        if result.modified_count > 0:
                            update_count += 1

                    print(f"-> ✅ {update_count} Laps updated.")

                except Exception as e:
                    print(f"-> ❌ Error: {e}")

        except Exception as e:
            print(f"Critical Error in {year}: {e}")

# 실행
update_pit_data(TARGET_YEARS)
print("\n✨ 모든 피트 스톱 데이터 업데이트가 완료되었습니다.")

✅ DB 연결 성공: f1_analytics_db


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info



🛠️ 2023년도 피트 데이터 업데이트 시작 (23 라운드)...
   Processing: R1 Bahrain Grand Prix... 

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


-> ✅ 102 Laps updated.
   Processing: R2 Saudi Arabian Grand Prix... 

...

-> ✅ 57 Laps updated.
   Processing: R21 São Paulo Grand Prix... 

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Driver 4 completed the race distance 00:00.010000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['4', '12', '1', '63', '81', '87', '30', '6', '27', '10', '23', '31', '55', '14', '43', '18', '22', '44', '16', '5']
core           INFO 	Loading data for Las Vegas Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


-> ✅ 77 Laps updated.
   Processing: R22 Las Vegas Grand Prix... 

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '63'
core           INFO 	Finished loading data for 20 drivers: ['1', '63', '12', '16', '55', '6', '27', '44', '31', '87', '14', '22', '10', '30', '43', '23', '5', '18', '4', '81']


-> ✅ 44 Laps updated.

✨ 모든 피트 스톱 데이터 업데이트가 완료되었습니다.
